In [257]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [258]:
dfx=pd.read_csv("Diabetes_XTrain.csv")
dfx

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,7,168,88,42,321,38.2,0.787,40
1,8,110,76,0,0,27.8,0.237,58
2,7,147,76,0,0,39.4,0.257,43
3,2,100,66,20,90,32.9,0.867,28
4,4,129,86,20,270,35.1,0.231,23
...,...,...,...,...,...,...,...,...
571,2,111,60,0,0,26.2,0.343,23
572,7,187,68,39,304,37.7,0.254,41
573,2,122,60,18,106,29.8,0.717,22
574,4,154,72,29,126,31.3,0.338,37


In [259]:
dfy=pd.read_csv("Diabetes_YTrain.csv")
dfy

,Outcome
0,1
1,0
2,1
3,1
4,0
...,...
571,0
572,1
573,0
574,0


In [260]:
X=dfx.values
Y=dfy.values

Y=Y.reshape((-1,))

# Normalisation

In [261]:
x_mean=X.mean(axis=0)
x_std=X.std(axis=0)
X=(X-x_mean)/x_std
X.shape

(576, 8)

In [262]:
dfx=pd.DataFrame(X,columns=dfx.columns)
dfx

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.974572,1.525281,0.945995,1.362427,2.244145,0.780366,0.907501,0.593634
1,1.275243,-0.316834,0.353934,-1.282400,-0.698144,-0.499183,-0.726400,2.110340
2,0.974572,0.858308,0.353934,-1.282400,-0.698144,0.928006,-0.666985,0.846418
3,-0.528785,-0.634440,-0.139449,-0.022959,0.126796,0.128288,1.145159,-0.417504
4,0.072558,0.286617,0.847318,-0.022959,1.776678,0.398962,-0.744224,-0.838811
...,...,...,...,...,...,...,...,...
571,-0.528785,-0.285073,-0.435480,-1.282400,-0.698144,-0.696037,-0.411503,-0.838811
572,0.974572,2.128732,-0.040773,1.173511,2.088323,0.718849,-0.675898,0.677895
573,-0.528785,0.064293,-0.435480,-0.148903,0.273453,-0.253116,0.699550,-0.923073
574,0.072558,1.080632,0.156581,0.543790,0.456773,-0.068566,-0.426356,0.340849


In [263]:
X=dfx.values

# Feature Selection

In [264]:
from sklearn.ensemble import RandomForestClassifier

In [351]:
alg=RandomForestClassifier()
alg.fit(X,Y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [352]:
alg.feature_importances_

array([0.08012317, 0.25757845, 0.08543246, 0.07090515, 0.0723811 ,
       0.17834143, 0.13162276, 0.12361547])

In [353]:
table=list(zip(dfx.columns,alg.feature_importances_))

In [354]:
best_features_df=pd.DataFrame(table,columns=['Feature','Score'])
best_features_df.sort_values(by='Score',inplace=True,ascending=False)
best_features_df

,Feature,Score
1,Glucose,0.257578
5,BMI,0.178341
6,DiabetesPedigreeFunction,0.131623
7,Age,0.123615
2,BloodPressure,0.085432
0,Pregnancies,0.080123
4,Insulin,0.072381
3,SkinThickness,0.070905


In [458]:
four_features=best_features_df.head(3)['Feature'].values
four_features

array(['Glucose', 'BMI', 'DiabetesPedigreeFunction', 'Age',
       'BloodPressure', 'Pregnancies'], dtype=object)

# Check The Model Performance

In [459]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [460]:
rfc=RandomForestClassifier()

In [461]:
#Without Feature Selection

score=cross_val_score(rfc,X,Y,cv=10)
print(score)
print(score.mean()*100)

[0.72413793 0.70689655 0.74137931 0.74137931 0.82758621 0.67241379
 0.75438596 0.73684211 0.78947368 0.75438596]
74.48880822746521


In [462]:
#With Feature Selection

dfx_2=dfx[four_features]
X2=dfx_2.values
X2

array([[ 1.52528095,  0.78036618,  0.907501  ,  0.59363371,  0.94599501,
         0.97457151],
       [-0.31683408, -0.49918316, -0.72639999,  2.11034006,  0.35393439,
         1.27524274],
       [ 0.85830827,  0.92800649, -0.66698541,  0.8464181 ,  0.35393439,
         0.97457151],
       ...,
       [ 0.06429317, -0.25311598,  0.69954996, -0.92307263, -0.43547977,
        -0.52878465],
       [ 1.0806325 , -0.06856559, -0.42635635,  0.34084932,  0.15658085,
         0.07255781],
       [-1.01556737,  0.01755792, -0.34614667, -0.33324239,  0.45261116,
        -0.82945588]])

In [463]:
score=cross_val_score(rfc,X2,Y,cv=10)
print(score)
print(score.mean()*100)

[0.74137931 0.70689655 0.75862069 0.77586207 0.79310345 0.67241379
 0.78947368 0.75438596 0.80701754 0.75438596]
75.53539019963702


# Split

In [464]:
from sklearn import model_selection as ms

In [465]:
X_train,X_test,Y_train,Y_test=ms.train_test_split(dfx,dfy,random_state=1)

In [466]:
Y_train=Y_train.values
Y_test=Y_test.values

Y_test=Y_test.reshape(-1,)
Y_train=Y_train.reshape(-1,)

In [467]:
X_train.shape,Y_train.shape,X_test.shape, Y_test.shape

((432, 8), (432,), (144, 8), (144,))

In [468]:
X_train=X_train[four_features]
X_train=X_train.values
X_train

array([[ 0.25485679,  0.30053518, -0.53033187, -0.7545497 ,  0.05790408,
         0.07255781],
       [-0.95204616, -1.0282276 , -0.86305353, -0.67028824, -0.43547977,
        -0.52878465],
       [ 1.33471733,  2.62587005,  0.82432058, -0.67028824,  0.35393439,
        -1.13012711],
       ...,
       [-0.66620073, -0.84367722, -0.67886833, -0.92307263, -3.39578288,
        -1.13012711],
       [ 2.44633847, -0.16699247, -0.96108759,  1.68903274,  0.05790408,
        -0.52878465],
       [-2.0001461 ,  0.11598479, -1.14527279,  0.67789518,  0.55128793,
         1.57591397]])

In [469]:
X_test=X_test[four_features]
X_test=X_test.values
X_test

array([[-1.26965220e+00, -6.96036908e-01, -5.00624581e-01,
         6.77895177e-01, -1.39449463e-01,  6.73900279e-01],
       [-4.43876497e-01,  9.28006491e-01,  3.66828308e-01,
        -9.23072630e-01,  5.79040769e-02, -1.13012711e+00],
       [ 2.16049303e+00,  1.97379201e+00, -1.02347290e+00,
         8.46418104e-01,  4.52611158e-01,  1.27524274e+00],
       [-9.52046162e-01, -9.17497372e-01, -6.90751242e-01,
        -7.54549703e-01,  5.51287928e-01, -5.28784649e-01],
       [-3.48594685e-01,  3.62051973e-01,  1.25804703e+00,
        -5.86026776e-01, -2.38126234e-01,  7.25578147e-02],
       [-1.89791665e-01,  1.15984791e-01, -6.64014680e-01,
         7.62156641e-01, -1.39449463e-01,  9.74571511e-01],
       [ 5.40702228e-01, -8.08689540e-02, -6.81839055e-01,
        -2.48980922e-01,  7.48641468e-01,  7.25578147e-02],
       [ 1.77936579e+00,  2.26715023e-01, -4.31323042e-02,
         2.11034006e+00,  1.04467178e+00,  1.27524274e+00],
       [-9.45098530e-02,  2.39018382e-01, -4.610

In [470]:
X_train.shape,Y_train.shape,X_test.shape, Y_test.shape

((432, 6), (432,), (144, 6), (144,))

# Using SKLearn Logistic Reg

In [471]:
from sklearn.linear_model import LogisticRegression as LR

In [472]:
alg=LR()

In [473]:
alg.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [474]:
alg.score(X_train,Y_train)

0.7939814814814815

In [475]:
alg.score(X_test,Y_test)

0.7430555555555556

### Adding Columns

In [476]:
ones=np.ones((X_train.shape[0],1))
X_train_new=np.hstack((ones,X_train))

In [477]:
X_train_new.shape

(432, 7)

In [478]:
ones=np.ones((X_test.shape[0],1))
X_test_new=np.hstack((ones,X_test))

In [479]:
X_test_new.shape

(144, 7)

### Using Logistic Regression

In [480]:
def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def hypo(X,theta):
    return sigmoid(np.dot(X,theta))

def error(X,y,theta):
    yp=hypo(X,theta)
    e=np.mean((y*np.log(yp))+((1-y)*np.log(1-yp)))
    return -e

def gradient(X,y,theta):
    M,N=X.shape
    yp=hypo(X,theta)
    grad=np.dot(y-yp,X)
    return -grad/M

def gradient_descent(X,y,lr=0.1,max_itr=500):
    M,N=X.shape
    theta=np.zeros((N,))
    error_list=[]
    
    for i in range(max_itr):
        e=error(X,y,theta)
        error_list.append(e)
        grad=gradient(X,y,theta)
        theta=theta-lr*grad
        
    return theta,error_list

def predict(X,theta):
    yp=hypo(X,theta)
    output=np.zeros(yp.shape)
    output[yp>=0.5]=1
    output=output.astype('int')
    return output

def accuracy(Y,Yp):
    Y=Y.astype('int')
    return np.mean(Y==Yp)*100

In [481]:
theta,error_list=gradient_descent(X_train_new,Y_train)

In [482]:
Y_train_pred=predict(X_train_new,theta)
accuracy(Y_train,Y_train_pred)

79.39814814814815

In [483]:
Y_test_pred=predict(X_test_new,theta)
accuracy(Y_test,Y_test_pred)

74.30555555555556

In [484]:
error_list

[0.6931471805599453,
 0.6808551878181348,
 0.6694348619589329,
 0.6588168866026881,
 0.6489365539297265,
 0.6397338227436296,
 0.6311532589081579,
 0.6231438883310086,
 0.6156589887315929,
 0.6086558419741608,
 0.6020954643276725,
 0.5959423279714534,
 0.5901640835740145,
 0.5847312908858909,
 0.5796171619856643,
 0.5747973200401041,
 0.570249575105803,
 0.5659537175293429,
 0.5618913288203206,
 0.5580456094106915,
 0.5544012224201141,
 0.5509441523762246,
 0.547661577756458,
 0.5445417561976447,
 0.5415739212411521,
 0.5387481895300359,
 0.5360554774397817,
 0.5334874261980919,
 0.531036334626462,
 0.5286950987133862,
 0.5264571573035484,
 0.5243164432578398,
 0.5222673395046642,
 0.5203046394633745,
 0.5184235113757543,
 0.5166194661313255,
 0.5148883282172085,
 0.5132262094635628,
 0.5116294852916966,
 0.5100947732040949,
 0.5086189132842472,
 0.5071989504996246,
 0.5058321186237658,
 0.5045158256135087,
 0.5032476402952116,
 0.5020252802296058,
 0.5008466006389347,
 0.4997095842924

# Testing The Data

In [393]:
xtest=pd.read_csv("Diabetes_Xtest.csv")
xtest

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0,180,90,26,90,36.5,0.314,35
1,2,93,64,32,160,38.0,0.674,23
2,2,114,68,22,0,28.7,0.092,25
3,13,76,60,0,0,32.8,0.180,41
4,1,80,74,11,60,30.0,0.527,22
...,...,...,...,...,...,...,...,...
187,10,108,66,0,0,32.4,0.272,42
188,3,130,64,0,0,23.1,0.314,22
189,0,162,76,36,0,49.6,0.364,26
190,12,84,72,31,0,29.7,0.297,46


In [394]:
# Normalisation:

xtest=(xtest-x_mean)/x_std
xtest=xtest[four_features]
#xtest=(xtest-xtest.mean(axis=0))/xtest.std(axis=0)
xtest

,Glucose,BMI,DiabetesPedigreeFunction,Age
0,1.906408,0.571209,-0.497654,0.172326
1,-0.856764,0.755759,0.571809,-0.838811
2,-0.189792,-0.388453,-1.157156,-0.670288
3,-1.396695,0.115985,-0.895732,0.677895
4,-1.269652,-0.228509,0.135111,-0.923073
...,...,...,...,...
187,-0.380355,0.066771,-0.622424,0.762157
188,0.318378,-1.077441,-0.497654,-0.923073
189,1.334717,2.182949,-0.349117,-0.586027
190,-1.142610,-0.265419,-0.548156,1.099202


In [395]:
xtest_new=xtest.values
ones=np.ones((xtest_new.shape[0],1))
xtest_new=np.hstack((ones,xtest_new))

In [396]:
xtest_new.shape

(192, 5)

In [397]:
y_test_pred=predict(xtest_new,theta)

In [398]:
y_test_pred

array([1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0])

In [399]:
df_res=pd.DataFrame(y_test_pred,columns=['Outcome'])
df_res

,Outcome
0,1
1,0
2,0
3,0
4,0
...,...
187,0
188,0
189,1
190,0


In [400]:
df_res.to_csv("diabetes_result.csv",index=None)